In [8]:
%reload_ext autoreload
%autoreload 2
from pathlib import Path
import wfdb
import sys
sys.path.append("..")

from util import list_patient_id
from preprocessing import match_channels

In [3]:
path_to_data = Path("/media/hdd1/i-care/training")

In [4]:
# For each patient load in EEG segment
for p in list_patient_id(path_to_data)[:1]:
    # Load in each segment of EEG from patient
    eeg_path = path_to_data / p
    for eeg_segment in sorted(set([seg.parent / seg.stem for seg in eeg_path.glob("*EEG.mat")]))[:1]:
        # Read in EEG segment, separating the signal and the channels
        eeg_record = wfdb.io.rdrecord(eeg_segment)
        eeg_signal = eeg_record.p_signal
        eeg_channels = eeg_record.sig_name
        # Remove excess channels and reorder the channels to a standardized order
        match_channels(eeg_signal, eeg_channels)

In [6]:
eeg_signal.shape

(1285000, 19)

In [11]:
eeg_record.sig_name

['Fp1',
 'Fp2',
 'F3',
 'F4',
 'C3',
 'C4',
 'P3',
 'P4',
 'O1',
 'O2',
 'F7',
 'F8',
 'T3',
 'T4',
 'T5',
 'T6',
 'Fz',
 'Cz',
 'Pz']